<a href="https://colab.research.google.com/github/AnuragBhattacharjee17/ConvolutionalNeuralNetwork-CNN-/blob/master/Cats_vs_Dogs_(Kaggle).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install keras.tuner

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for keras.tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=f434d5ef2e2e9aa28472f76d6710adb5457c472933a41edbe1aea5361b6a8956
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=56dc58629015f782f619d2d5c315f8a1e762f105c42c026c1fd1901ea6495f83
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras.tuner terminaltables


In [0]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import cv2
from os import listdir
from os.path import join
from os import listdir, rmdir
from shutil import move
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from numpy import asarray
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip -q "/content/drive/My Drive/kagglecatsanddogs_3367a.zip"

In [0]:
folder = "/content/PetImages/"
cat = "/content/PetImages/Cat/"
dog = "/content/PetImages/Dog/"

In [0]:
#Cat Rename
i=0
for filename in os.listdir(cat):
  dst="cat"+str(i)+".jpg"
  src= cat + filename
  dst= cat + dst
  os.rename(src,dst)
  i = i+1

#Dog Rename
i=0
for filename in os.listdir(dog):
  dst="dog"+str(i)+".jpg"
  src= dog + filename
  dst= dog + dst
  os.rename(src,dst)
  i = i+1

In [0]:
for filename in listdir(join(folder, cat)):
    move(join(folder, cat, filename), join(folder, filename))

for filename in listdir(join(folder, dog)):
    move(join(folder, dog, filename), join(folder, filename))

In [0]:
rmdir(cat)
rmdir(dog)

In [0]:
os.remove("/content/PetImages/cat11887.jpg")
os.remove("/content/PetImages/cat8748.jpg")
os.remove("/content/PetImages/dog1211.jpg")
os.remove("/content/PetImages/dog8748.jpg")

In [0]:
path, dirs, files = next(os.walk("/content/PetImages/"))
file_count = len(files)
file_count

24998

In [0]:
photos, labels = list(), list()
# enumerate files in the directory
for file in listdir(folder):
  if file.startswith("cat"):
    count=1
  else:
    count=2
  photo = load_img(folder + file, target_size=(150, 150))
  photo = img_to_array(photo)
	# store
  photos.append(photo)
  labels.append(count)

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:784: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:784: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:784: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:784: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:784: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

In [0]:
len(photos),len(labels)

(24998, 24998)

In [0]:
photos[0].shape

(150, 150, 3)

In [0]:
# convert to a numpy arrays
photos = asarray(photos)
labels = asarray(labels)
print(photos.shape, labels.shape)

(24998, 150, 150, 3) (24998,)


In [0]:
labels=to_categorical(labels)

In [0]:
photos=photos/255.0

In [0]:
labels.shape

(24998, 3)

In [0]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch


def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=8),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(150,150,3)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=8),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=8),
        activation='relu'
    ),
    keras.layers.Dense(3, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [0]:
tuner = RandomSearch(
    build_model,
    objective='val_acc',
    max_trials=10,
    directory='my_dir',
    project_name='CatVsDogs')

In [0]:
tuner.search(photos,labels,epochs=3,validation_split=0.05)

Train on 23748 samples, validate on 1250 samples
Epoch 1/3
23748/23748 [==============================] - 61s 3ms/sample - loss: 4.0245 - acc: 0.4986 - val_loss: 0.6936 - val_acc: 0.4840
Epoch 2/3
23748/23748 [==============================] - 54s 2ms/sample - loss: 0.6941 - acc: 0.5009 - val_loss: 0.6928 - val_acc: 0.5160
Epoch 3/3
23748/23748 [==============================] - 50s 2ms/sample - loss: 0.6939 - acc: 0.4998 - val_loss: 0.6955 - val_acc: 0.4840


Train on 23748 samples, validate on 1250 samples
Epoch 1/3
23748/23748 [==============================] - 43s 2ms/sample - loss: 5.3907 - acc: 0.4965 - val_loss: 0.6928 - val_acc: 0.5160
Epoch 2/3
23748/23748 [==============================] - 40s 2ms/sample - loss: 0.6940 - acc: 0.4987 - val_loss: 0.6935 - val_acc: 0.4840
Epoch 3/3
23748/23748 [==============================] - 40s 2ms/sample - loss: 0.6943 - acc: 0.4930 - val_loss: 0.6947 - val_acc: 0.4840


Train on 23748 samples, validate on 1250 samples
Epoch 1/3
23748/23748 [==============================] - 37s 2ms/sample - loss: 6.4940 - acc: 0.5021 - val_loss: 0.6932 - val_acc: 0.5160
Epoch 2/3
23748/23748 [==============================] - 35s 1ms/sample - loss: 0.6942 - acc: 0.4962 - val_loss: 0.6935 - val_acc: 0.4840
Epoch 3/3
23748/23748 [==============================] - 35s 1ms/sample - loss: 0.6939 - acc: 0.4976 - val_loss: 0.6934 - val_acc: 0.4840


Train on 23748 samples, validate on 1250 samples
Epoch 1/3
23748/23748 [==============================] - 55s 2ms/sample - loss: 36.3543 - acc: 0.5006 - val_loss: 0.6940 - val_acc: 0.5160
Epoch 2/3
23748/23748 [==============================] - 48s 2ms/sample - loss: 0.6950 - acc: 0.4983 - val_loss: 0.6928 - val_acc: 0.5160
Epoch 3/3
23748/23748 [==============================] - 48s 2ms/sample - loss: 0.6939 - acc: 0.4997 - val_loss: 0.6927 - val_acc: 0.5160


Train on 23748 samples, validate on 1250 samples
Epoch 1/3
23748/23748 [==============================] - 46s 2ms/sample - loss: 8.1800 - acc: 0.4977 - val_loss: 0.6928 - val_acc: 0.5160
Epoch 2/3
23748/23748 [==============================] - 42s 2ms/sample - loss: 0.6943 - acc: 0.5032 - val_loss: 0.6941 - val_acc: 0.4840
Epoch 3/3
23748/23748 [==============================] - 42s 2ms/sample - loss: 0.6936 - acc: 0.5037 - val_loss: 0.6981 - val_acc: 0.4840


Train on 23748 samples, validate on 1250 samples
Epoch 1/3
23748/23748 [==============================] - 47s 2ms/sample - loss: 0.8252 - acc: 0.5853 - val_loss: 0.6881 - val_acc: 0.5832
Epoch 2/3
23748/23748 [==============================] - 51s 2ms/sample - loss: 0.6123 - acc: 0.6713 - val_loss: 0.6379 - val_acc: 0.6536
Epoch 3/3
23748/23748 [==============================] - 47s 2ms/sample - loss: 0.4825 - acc: 0.7679 - val_loss: 0.5819 - val_acc: 0.7208


Train on 23748 samples, validate on 1250 samples
Epoch 1/3
23748/23748 [==============================] - 60s 3ms/sample - loss: 2.2293 - acc: 0.5013 - val_loss: 0.6934 - val_acc: 0.4840
Epoch 2/3
23748/23748 [==============================] - 52s 2ms/sample - loss: 0.6950 - acc: 0.4971 - val_loss: 0.6947 - val_acc: 0.5160
Epoch 3/3
23748/23748 [==============================] - 46s 2ms/sample - loss: 0.6949 - acc: 0.5031 - val_loss: 0.6947 - val_acc: 0.4840


Train on 23748 samples, validate on 1250 samples
Epoch 1/3
23748/23748 [==============================] - 60s 3ms/sample - loss: 0.7363 - acc: 0.5974 - val_loss: 0.6532 - val_acc: 0.6160
Epoch 2/3
23748/23748 [==============================] - 55s 2ms/sample - loss: 0.5949 - acc: 0.6847 - val_loss: 0.7167 - val_acc: 0.5864
Epoch 3/3
23748/23748 [==============================] - 59s 2ms/sample - loss: 0.4678 - acc: 0.7779 - val_loss: 0.6554 - val_acc: 0.6696


Train on 23748 samples, validate on 1250 samples
Epoch 1/3
23748/23748 [==============================] - 52s 2ms/sample - loss: 3.4738 - acc: 0.5006 - val_loss: 0.6928 - val_acc: 0.5160
Epoch 2/3
 2848/23748 [==>...........................] - ETA: 41s - loss: 0.6938 - acc: 0.4919

KeyboardInterrupt: ignored

In [0]:
model=tuner.get_best_models(num_models=1)[0]

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 146, 146, 56)      4256      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 144, 144, 56)      28280     
_________________________________________________________________
flatten (Flatten)            (None, 1161216)           0         
_________________________________________________________________
dense (Dense)                (None, 104)               120766568 
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 315       
Total params: 120,799,419
Trainable params: 120,799,419
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(photos, labels, epochs=5, validation_split=0.2, initial_epoch=3)

In [0]:
test_loss, test_acc = model.evaluate(photos, labels)
print('test_acc:', test_acc*100)

24998/24998 [==============================] - 17s 682us/sample - loss: 0.1695 - acc: 0.9634
test_acc: 96.33570909500122
